<a href="https://colab.research.google.com/github/Gaurav37/A_star/blob/master/Webcam_YOLO_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/opencv/opencv.git

Cloning into 'opencv'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 281418 (delta 0), reused 6 (delta 0), pack-reused 281395
Receiving objects: 100% (281418/281418), 473.55 MiB | 26.27 MiB/s, done.
Resolving deltas: 100% (196561/196561), done.
Checking out files: 100% (6654/6654), done.


In [ ]:
!mkdir Video

In [ ]:
!pip install ffmpeg-python


from IPython.display import HTML, Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

video_file_test = '/content/Video/osy_test.mp4' 
  

VIDEO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var my_btn_txt = document.createTextNode("Press to start recording");

my_btn.appendChild(my_btn_txt);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, videoStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  videoStream = stream;
  var options = {  
    mimeType : 'video/webm;codecs=vp9'  
  };            
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('video');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({video: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      videoStream.getVideoTracks()[0].stop();
      recordButton.innerText = "Saving the recording... Please wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def start_webcam():
  js = Javascript('''
    async function startWebcam() {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      return;

    }
    ''')
  
  display(js)
  data = eval_js('startWebcam()')
  
    
start_webcam()

def get_video():
  display(HTML(VIDEO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  return binary

<IPython.core.display.Javascript object>

In [ ]:
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov2-tiny.cfg


--2020-11-02 19:28:20--  https://github.com/pjreddie/darknet/blob/master/cfg/yolov2-tiny.cfg
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘yolov2-tiny.cfg’

yolov2-tiny.cfg         [ <=>                ] 114.23K  --.-KB/s    in 0.08s   

2020-11-02 19:28:20 (1.38 MB/s) - ‘yolov2-tiny.cfg’ saved [116968]



In [ ]:
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names

--2020-11-02 19:32:06--  https://github.com/pjreddie/darknet/blob/master/data/coco.names
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘coco.names’

coco.names              [ <=>                ] 100.30K  --.-KB/s    in 0.08s   

2020-11-02 19:32:06 (1.27 MB/s) - ‘coco.names’ saved [102710]



In [ ]:
import cv2
import numpy as np

net =cv2.dnn.readNet('yolov2-tiny.weights', 'yolov2-tiny.cfg')
classes=[]
with open('coco.names','r') as f:
  classes=f.read().splitlines()

cap=cv2.VideoCapture()
#img=cv2.imread('image.jpg')
while True:
  _, img =cap.read() 
  height,width, _=img.shape

  blob=cv2.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True, crop=False)

  '''for b in blob:
    for n, img_blob in enumerate(b):
      cv2.imshow(str(n),img_blob)
  '''

  net.setInput(blob)
  output_layers_names=net.getUnconnectedOutputLayersNames()
  layerOutputs=net.forward(output_layers_names)

  boxes=[]
  confidences=[]
  class_ids=[]

  for output in layerOutputs:
    for detection in output:
      scores= detection[5:]
      class_ids = np.argmax(scores)
      confidence=scores[class_id]
      if confidence>0.5:
        center_x=int(detection[0]*width)
        center_y= int(detection[1]*height)
        w= int(detection[3]*width)
        h= int(detection[3]*width)

        x=int(center_x -w/2)
        y = int(center_y -h/2)

        boxes.append([x,y,w,h])
        confidences.append((float(confidence)))
        class_ids.append(class_id)

  #print(len(boxes))
  indexes= cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.4)
  #print(indexes.flatten)
  font=cv2.FONT_HERSHEY_PLAIN
  colors=np.random.uniform(0,255,size=(len(boxes),3))

  for i in indexes.flatten():
    x,y,w,h=boxes[i]
    label=str(classes[class_ids[i]])
    confidence=str(round(confidences[i],2))
    color=colors[i]
    cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
    cv2.putText(img,label + "" + confidence, (x,y+20),font,2,(255,255,255),2)


  cv2.imshow('Image',img)
  key=cv2.waitKey(1)
  if key==27:
    break
cap.release()
cv2.destroyAllWindows()